# Wordly
## Experiment 2016-12-21

Initial experiments with n-gram model. 

In [7]:
import sys; sys.path.append('../../src')
import wordly
reload(wordly)
dir(wordly)

['NgramModel',
 '__builtins__',
 '__doc__',
 '__file__',
 '__name__',
 '__package__',
 '__path__',
 'ngram']

In [8]:
m = wordly.NgramModel(2) # bigram
#m.train(s)
m

<wordly.ngram.NgramModel instance at 0x00000000090E1088>

In [9]:
dir(m)

['__doc__',
 '__init__',
 '__module__',
 '__str__',
 'd',
 'get_random',
 'increment',
 'load',
 'n',
 'predict',
 'save',
 'train']